In [42]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns

import requests
import csv

import warnings

In [43]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [44]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [52]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Define the ticker symbols as a list; eg. TSM,MSFT,AMZN
# ticker_symbols = input("Enter stock tickers separated by commas:") 
# ticker_symbols = ticker_symbols.split(',')
ticker_symbols = [
    'TSM'
]

# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict = {}

# PE TTM Valuation

In [53]:
# Daily quote section
for symbol in ticker_symbols:


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':

            

            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': f'{symbol}'
                    }
                ,inplace=True
                )
            
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].round(2)


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': f'{symbol}'
            }
        ,inplace=True
        )

    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].round(2)



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue

            annualEPS_df[f'{symbol}_PE'] = Monthly_stock_df[f'{symbol}'].values / annualEPS_df['reportedEPS'].values
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE"].mean().round(2)
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df[f"{symbol}_PE"]).round(2)
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[f'{col}'] = qtrEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from nasdaq webscrapping




    # Consolidated section
    df_stock_consolidate = Daily_stock_df.head(window_days)


    df_stock_consolidate_date = df_stock_consolidate.index
    for i in df_stock_consolidate_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate']<= i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        df_stock_consolidate.loc[i, f"{symbol}_EPS_TTM"] = EPS_TTM

        if i == max(df_stock_consolidate.index):
            EPS_latest_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        df_stock_consolidate[f"{symbol}_EPS_latest_projected"] = EPS_latest_projected
    
    # stock's stats
    df_stock_consolidate[f"{symbol}_PE_TTM"] = (df_stock_consolidate[symbol] / df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_avg"] = df_stock_consolidate[f"{symbol}_PE_TTM"].mean().round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_std"] = np.std(df_stock_consolidate[f"{symbol}_PE_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] + df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] - df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    df_stock_consolidate[f"{symbol}_relative_valuation_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_+"], df_stock_consolidate[f"{symbol}_relative_valuation_-"]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_relative_valuation_projected_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_projected_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_projected_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_projected_+"], df_stock_consolidate[f"{symbol}_relative_valuation_projected_-"]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_{window_days}_price_min"] = df_stock_consolidate[symbol].min().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_max"] = df_stock_consolidate[symbol].max().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_avg"] = df_stock_consolidate[symbol].mean().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_std"] = np.std(df_stock_consolidate[symbol]).round(2)

    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"].values[0]
    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"].values[0]
    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    conditions = [
    (df_stock_consolidate[f"{symbol}"] < df_stock_consolidate[f"{symbol}_relative_valuation_-"]),
    (df_stock_consolidate[f"{symbol}"] > df_stock_consolidate[f"{symbol}_relative_valuation_+"]),
    ((df_stock_consolidate[f"{symbol}"] >= df_stock_consolidate[f"{symbol}_relative_valuation_-"]) & (df_stock_consolidate[f"{symbol}"] <= df_stock_consolidate[f"{symbol}_relative_valuation_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    df_stock_consolidate[f"{symbol}_curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        df_stock_consolidate.loc[condition, f"{symbol}_price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    f"{symbol}"
    # ,f"{symbol}_EPS_TTM"
    # ,f"{symbol}_EPS_latest_projected"
    ,f"{symbol}_PE_TTM"
    ,f"{symbol}_PE_TTM_avg"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_avg"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"
    ,f"{symbol}_relative_valuation_+"
    ,f"{symbol}_relative_valuation_-"
    ,f"{symbol}_relative_valuation_median"
    ,f"{symbol}_relative_valuation_projected_+"
    ,f"{symbol}_relative_valuation_projected_-"
    ,f"{symbol}_relative_valuation_projected_median"
    ,f"{symbol}_price_valuation_assessment"
    ]

    ticker_dict[f'{symbol}'] = df_stock_consolidate[selected_cols]

    # Addtional setting session
    # Set display options to show all rows and columns
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    warnings.filterwarnings("ignore")
    pd.options.mode.copy_on_write = True

In [54]:
ticker_dict['TSM']

,TSM,TSM_PE_TTM,TSM_PE_TTM_avg,TSM_PE_5yr_avg,TSM_PE_5yr_volatility_+,TSM_PE_5yr_volatility_-,TSM_relative_valuation_+,TSM_relative_valuation_-,TSM_relative_valuation_median,TSM_relative_valuation_projected_+,TSM_relative_valuation_projected_-,TSM_relative_valuation_projected_median,TSM_price_valuation_assessment
2024-06-21,173.96,33.07,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-20,175.38,33.34,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-18,179.69,34.16,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-17,177.24,33.70,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-14,172.51,32.80,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-13,172.91,32.87,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-12,172.98,32.89,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-11,165.71,31.50,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-10,168.16,31.97,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued
2024-06-07,164.39,31.25,27.59,22.66,30.05,15.27,158.59,131.66,148.95,166.13,137.92,152.02,overvalued


# IPO

In [ ]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=IPO_CALENDAR&apikey={alpha_vantage_api_key}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

    IPO_df = pd.DataFrame(columns=my_list[0]
                          ,data=my_list[1::])

In [ ]:
IPO_df

,symbol,name,ipoDate,priceRangeLow,priceRangeHigh,currency,exchange
0,FSHPU,Flag Ship Acquisition Corp. Unit,2024-06-18,0,0,USD,NASDAQ
1,CCIXW,Churchill Capital Corp IX Warrant,2024-06-21,0,0,USD,NASDAQ
2,CCIX,Churchill Capital Corp IX Ordinary Shares,2024-06-21,0,0,USD,NASDAQ
